<a href="https://colab.research.google.com/github/neurosxx/Descriptive-Stats-Phase-2-/blob/main/SQL_Subqueries_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [1]:
import sqlite3
import pandas as pd

In [2]:
!wget https://github.com/learn-co-curriculum/dsc-one-to-many-and-many-to-many-joins/raw/refs/heads/master/data.sqlite

--2024-10-13 18:38:07--  https://github.com/learn-co-curriculum/dsc-one-to-many-and-many-to-many-joins/raw/refs/heads/master/data.sqlite
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/learn-co-curriculum/dsc-one-to-many-and-many-to-many-joins/refs/heads/master/data.sqlite [following]
--2024-10-13 18:38:07--  https://raw.githubusercontent.com/learn-co-curriculum/dsc-one-to-many-and-many-to-many-joins/refs/heads/master/data.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344064 (336K) [application/octet-stream]
Saving to: ‘data.sqlite’

data.sqlite         100%[===================>] 336.00

In [3]:
conn = sqlite3.Connection('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">


In [4]:
q = """
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN
                      (SELECT customerNumber
                      FROM orders
                      JOIN customers USING(customerNumber)
                      WHERE orderDate = '2003-01-31')
;
"""
pd.read_sql(q,conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">


In [5]:
q = """
SELECT
  COUNT(orderNumber) AS number_of_Orders, p.productName,
  SUM(o.quantityOrdered) AS totalUnitsSold
FROM orderdetails AS o
JOIN products AS p USING(productCode)
GROUP by productName
Order BY totalUnitsSold DESC



;
"""
pd.read_sql(q,conn)
df = pd.read_sql(q,conn)
df.head(10)

# Notice from select statement p.product name is only column, others are aggregate functions,
#That's why you group by product name here.

,number_of_Orders,productName,totalUnitsSold
0,53,1992 Ferrari 360 Spider red,1808
1,28,1937 Lincoln Berline,1111
2,28,American Airlines: MD-11S,1085
3,28,1941 Chevrolet Special Deluxe Cabriolet,1076
4,28,1930 Buick Marquette Phaeton,1074
5,28,1940s Ford truck,1061
6,28,1969 Harley Davidson Ultimate Chopper,1057
7,28,1957 Chevy Pickup,1056
8,28,1964 Mercedes Tour Bus,1053
9,27,1956 Porsche 356A Coupe,1052


**Making subqeury below from previous example**
<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">


In [6]:
q = """
SELECT
  COUNT(o.orderNumber) AS number_of_Orders,
  p.productName,
  SUM(o.quantityOrdered) AS totalUnitsSold
FROM orderdetails AS o
JOIN products AS p USING(productCode)
WHERE o.productCode IN (
  SELECT productCode
  FROM products
)
GROUP BY p.productName  -- Group by product name
ORDER BY SUM(o.quantityOrdered) DESC;
"""
pd.read_sql(q,conn)
df = pd.read_sql(q,conn)
df.head(10)

# This is subquery version of above!

,number_of_Orders,productName,totalUnitsSold
0,53,1992 Ferrari 360 Spider red,1808
1,28,1937 Lincoln Berline,1111
2,28,American Airlines: MD-11S,1085
3,28,1941 Chevrolet Special Deluxe Cabriolet,1076
4,28,1930 Buick Marquette Phaeton,1074
5,28,1940s Ford truck,1061
6,28,1969 Harley Davidson Ultimate Chopper,1057
7,28,1957 Chevy Pickup,1056
8,28,1964 Mercedes Tour Bus,1053
9,27,1956 Porsche 356A Coupe,1052


In [7]:
q = """
SELECT
  productName,
  SUM(quantityOrdered) AS totalSales
FROM orderdetails
JOIN products USING(productCode)
GROUP BY productName
HAVING totalSales > (
SELECT AVG(totalSales)
FROM(
  SELECT SUM(quantityOrdered) AS totalSales
  FROM orderdetails
  GROUp BY productCode
) AS sales_by_product


)
;
"""
pd.read_sql(q,conn)
df = pd.read_sql(q,conn)
df.head(10)


,productName,totalSales
0,18th century schooner,1011
1,1900s Vintage Tri-Plane,1009
2,1904 Buick Runabout,990
3,1912 Ford Model T Delivery Wagon,991
4,1913 Ford Model T Speedster,1038
5,1917 Maxwell Touring Car,992
6,1926 Ford Fire Engine,998
7,1928 British Royal Navy Airplane,972
8,1928 Ford Phaeton Deluxe,972
9,1930 Buick Marquette Phaeton,1074



<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [8]:
q = """
SELECT productName, COUNT(DISTINCT customerNumber) AS numPurchasers
FROM customers
JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
JOIN products USING(productCode)
GROUP BY productName
ORDER BY numPurchasers DESC
;
"""
pd.read_sql(q,conn)
df = pd.read_sql(q,conn)
df.head(10)

# We use COUNT here because it's counting the unique values for customerNumber Then you're grouping by productName.

# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

,productName,numPurchasers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
5,HMS Bounty,26
6,Corsair F4U ( Bird Cage),26
7,American Airlines: MD-11S,26
8,1999 Yamaha Speed Boat,26
9,1996 Moto Guzzi 1100i,26



<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [9]:
# Make sure use DISTINCT.
q = """
SELECT
DISTINCT employeeNumber,
                  firstName,
                  lastName,
                  o.city


FROM employees AS e
JOIN offices AS o USING(officeCode)
JOIN customers AS c
  ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
WHERE productCode IN (
SELECT productCode
FROM products
JOIN orderdetails
  USING(productCode)
JOIN orders
  USING(orderNumber)
GROUP BY productCode
HAVING COUNT(DISTINCT customerNumber) < 20

)
;
"""
pd.read_sql(q,conn)
df = pd.read_sql(q,conn)
df.head(10)

## THE TREND for this stuff is to join table on the key that is almost same,
# such as the Employee table joining ON salesRepEmployeeNumber (ALMOST SAME!)
# ALL the other tables join on same Key *

,employeeNumber,firstName,lastName,city
0,1370,Gerard,Hernandez,Paris
1,1501,Larry,Bott,London
2,1337,Loui,Bondur,Paris
3,1166,Leslie,Thompson,San Francisco
4,1286,Foon Yue,Tseng,NYC
5,1612,Peter,Marsh,Sydney
6,1611,Andy,Fixter,Sydney
7,1401,Pamela,Castillo,Paris
8,1621,Mami,Nishi,Tokyo
9,1323,George,Vanauf,NYC


In [10]:
q = """
SELECT
    DISTINCT employeeNumber,
    officeCode,
    o.city,
    firstName,
    lastName
FROM employees AS e
JOIN offices AS o
    USING(officeCode)
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
WHERE productCode IN (
    SELECT productCode
    FROM products
    JOIN orderdetails
        USING(productCode)
    JOIN orders
        USING(orderNumber)
    GROUP BY productCode
    HAVING COUNT(DISTINCT customerNumber) < 20
)
;
"""

pd.read_sql(q, conn)

##### THIS IS THE EXAMPLE FROM SOLUTION ####

,employeeNumber,officeCode,city,firstName,lastName
0,1370,4,Paris,Gerard,Hernandez
1,1501,7,London,Larry,Bott
2,1337,4,Paris,Loui,Bondur
3,1166,1,San Francisco,Leslie,Thompson
4,1286,3,NYC,Foon Yue,Tseng
5,1612,6,Sydney,Peter,Marsh
6,1611,6,Sydney,Andy,Fixter
7,1401,4,Paris,Pamela,Castillo
8,1621,5,Tokyo,Mami,Nishi
9,1323,3,NYC,George,Vanauf


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K


<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

In [37]:
q = """
SELECT
  employeeNumber,
  firstName,
  lastName,
  COUNT(DISTINCT customerNumber) AS numCustomers
FROM employees AS e
JOIN customers AS c
  ON e.employeeNumber=c.salesRepEmployeeNumber
GROUP BY employeeNumber
HAVING AVG(creditLimit) > 15000
ORDER BY numCustomers
"""
pd.read_sql(q,conn)

### ORDER BY numCustomers is not in solution,  ask if right!

,employeeNumber,firstName,lastName,numCustomers
0,1611,Andy,Fixter,5
1,1612,Peter,Marsh,5
2,1621,Mami,Nishi,5
3,1165,Leslie,Jennings,6
4,1166,Leslie,Thompson,6
5,1188,Julie,Firrelli,6
6,1216,Steve,Patterson,6
7,1337,Loui,Bondur,6
8,1702,Martin,Gerard,6
9,1286,Foon Yue,Tseng,7


In [34]:
q = """
SELECT
  employeeNumber,
  firstName,
  lastName,
  COUNT(DISTINCT customerNumber) AS distinctCustomers
FROM employees AS e
JOIN customers AS c
  ON e.employeeNumber=c.salesRepEmployeeNumber
WHERE creditLimit > 15000
GROUP BY employeeNumber
ORDER BY employeeNumber ASC

"""
pd.read_sql(q,conn)

# THIS ONE IS YOUR ATTEMPT, good work on it :)
# Needed to use AVG for creditLimit column.
#  ---- AVG function was hinted at in directions!

,employeeNumber,firstName,lastName,distinctCustomers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,5
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,6
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!